# Introduction

In this notebook we will consider a general approach to link prediction described in the paper "[Link Prediction Based on Graph Neural Networks](https://www.researchgate.net/profile/Muhan-Zhang/publication/323443864_Link_Prediction_Based_on_Graph_Neural_Networks/links/5dc113364585151435e9382a/Link-Prediction-Based-on-Graph-Neural-Networks.pdf)."


The basic idea here is to extract a subgraph around a link to be detected, and then classify the subgraph. The nodes in these subgraphs can have a feature vector built from attributes or embeddings obtained via an encoder (or both). This effectively translates a link prediction problem into a graph prediction problem.



# Azure setup
AML environemt setup and vaialbels

In [1]:
from azureml.core import Workspace, Environment, Experiment, ComputeTarget


In [2]:
EXPERIMENT_NAME = os.environ.get("EXPERIMENT_NAME", "link-prediction-train")
#CONDA_PATH = os.environ.get("CONDA_PATH", "../.azureml/score.yml")
COMPUTE_NAME = os.environ.get("COMPUTE_NAME", "devchamlc01") #mt
MODEL_NAME = os.environ.get("MODEL_NAME", "GNN-link-prediction")
MODEL_VERSION = os.environ.get("MODEL_VERSION", "1")


print("EXPERIMENT_NAME {}".format(EXPERIMENT_NAME))
#print("CONDA_PATH {}".format(CONDA_PATH))
print("COMPUTE_NAME {}".format(COMPUTE_NAME))
print("MODEL_NAME {}".format(MODEL_NAME))
print("MODEL_VERSION {}".format(MODEL_VERSION))

EXPERIMENT_NAME link-prediction-train
COMPUTE_NAME devchamlc01
MODEL_NAME GNN-link-prediction
MODEL_VERSION 1


In [3]:
ws = Workspace.from_config()
#ws = Workspace.get(name="mazcacnpeamlmtaml01",
#               subscription_id='046f49cd-89e9-495b-ae8d-a90fc8173ab3',
#               resource_group='maz-cac-aml-wstn-mt-rg')

print("Found workspace {} at location {}".format(ws.name, ws.location))

compute_target = ComputeTarget(ws, COMPUTE_NAME)

Found workspace mazcacdevchaml01 at location canadacentral


# package requirments
 install pytorch, go to https://pytorch.org/. It has instructions for how to install pytorch with cuda support.

# pytorch_geometric

In [4]:
# Please visit https://github.com/rusty1s/pytorch_geometric#pip-wheels for lastest installation instruction

!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html -U
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html -U
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html -U
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html -U
!pip install torch-geometric -U

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html


# Dataset
load dataset

In [6]:
import torch
from torch_geometric.datasets import PPI
from torch_geometric.utils import train_test_split_edges

# Load dataset 
PPI.url = "https://data.dgl.ai/dataset/ppi.zip" #  Workaround for wrong URL in pytorch geometric
dataset_ppi = PPI(root="./tmp/ppi") 

# For simplicity, pich the largest graph out of the dataset
data = max(dataset_ppi, key= lambda x:x.num_nodes) 

# Remove properties related to node-labeling (not needed here)
data.train_mask = data.val_mask = data.test_mask = data.y = None

# Create train/val/test split
data = train_test_split_edges(data, val_ratio=0.25, test_ratio=0.25,)
#data.x = torch.ones([data.x.shape[0], 2])

OSError: dlopen(/opt/anaconda3/lib/python3.8/site-packages/torch_sparse/_convert_cpu.so, 6): Symbol not found: __ZN2at5emptyEN3c108ArrayRefIxEENS0_13TensorOptionsENS0_8optionalINS0_12MemoryFormatEEE
  Referenced from: /opt/anaconda3/lib/python3.8/site-packages/torch_sparse/_convert_cpu.so
  Expected in: /opt/anaconda3/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib
 in /opt/anaconda3/lib/python3.8/site-packages/torch_sparse/_convert_cpu.so

# SEAL algorithm
The SEAL algorithm requires the extraction of subgraphs enclosing links, as well as the distance every node in the subgraph to each of the edge nodes. This functionality is not yet provided by Pytorch-Geometric, so we will using networkx for this purpose.

In [ ]:
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx

# Create a Data object using on only the positive training edges
data_pos = Data(edge_index=data.train_pos_edge_index, num_nodes=data.num_nodes)

# Convert this graph to networkx format
G_train_pos=to_networkx(data_pos).to_undirected()

In the SEAL link prediction framework, the nodes in the edge-enclosing subgraph are assigned a structural label according to their distance from the node-pair adjacent to the edge being considered. This label, called double radius, for node $i$ in the subgraph is defined by

$$ f(i) = 1 + min(d_x,d_y) + (d/2)[(d/2)+(d\%2)-1]$$
where $x$ and $y$ are the nodes adjacent to the considered edge, $d_x$ is the distance of node $i$ to $x$, $d_y$ is the distance of node $i$ to $y$ and $d= d_x+d_y$.

If $d_x = \infty$ or $d_y = \infty$ in the subgraph, they get a label of 0. The double radius of nodes $x$ and $y$ is 1. This helps identify the structural importance of nodes $x$ and $y$.

In [ ]:
def double_radius(d_x, d_y):
    if (d_x==0) or (d_y==0):
        return 1
    
    if np.isinf(d_x) or np.isinf(d_y):
        return 0
    d = d_x + d_y
    dr = 1 + min(d_x,d_y) + (d//2) * ( d//2 + d%2 -1 )
    return dr

We need to obtain n-hop subgraphs around each node. We will use n_hop=1.



In [ ]:
def create_ego_graphs(G, n_hops):
    dict_ego_graphs= {}
    for v in G.nodes():
        dict_ego_graphs[v] = nx.ego_graph(G,v, n_hops)
    return dict_ego_graphs
  
node_to_nhop_subgraphs = create_ego_graphs(G=G_train_pos,n_hops=1)

Once we have the n_hop subgraphs around each node, we can easily compute the double radius of the nodes in each edge-enclosing subgraph. (This is of course trivial for n_hop=1).

In [ ]:
import numpy as np

def get_edge_to_double_radii(node_to_nhop_subgraphs, edge, n_hops):
    v_x, v_y = edge

    # Compute distance from d_x to each node in subgraph
    subgraph_x = node_to_nhop_subgraphs[v_x]
    d_x = nx.single_source_shortest_path_length(subgraph_x, v_x, cutoff=n_hops)
    
    # Compute distance from d_y to each node in subgraph
    subgraph_y = node_to_nhop_subgraphs[v_y]
    d_y = nx.single_source_shortest_path_length(subgraph_y, v_y, cutoff=n_hops)
    
    # Get the union of the node ids for the two subgraphs
    nodes_subgraph = set(d_x.keys()) | set(d_y.keys())

    # Compute the double radius for each node in the union of the two subgraphs
    double_radii = {}

    for v_n in nodes_subgraph:
        d_x_to_n = d_x.get(v_n, np.inf)
        d_y_to_n = d_y.get(v_n, np.inf)
        double_radii[v_n] = double_radius(d_x_to_n, d_y_to_n)
    
    return double_radii

We will store the double radii for each edge-enclosing subgraph



In [ ]:
# helper function to get the double radii
def get_double_radii(edge_list, node_to_nhop_subgraph, n_hops):
    edge_to_double_radii = {}

    for edge in edge_list:
        if edge[1]>edge[0]: # Only get enclosing subgraph once
            double_radii=get_edge_to_double_radii(node_to_nhop_subgraphs,edge, n_hops=n_hops)
            edge_to_double_radii[tuple(edge)] = double_radii
    return edge_to_double_radii

In [ ]:
edge_to_double_radii_train_pos = get_double_radii(data.train_pos_edge_index.T.numpy(), node_to_nhop_subgraphs, 1)



We will now build the same subgraphs for negative samples. For the sake of speed, we will only use one set of negative training edge samples. We will also have to create subgraphs for the validation and testing sets



In [ ]:

from torch_geometric.utils import negative_sampling
neg_edge_index = negative_sampling(edge_index=data.train_pos_edge_index,
                                    num_nodes=data.x.size(0))

edge_to_double_radii_train_neg = get_double_radii(neg_edge_index.T.numpy(), node_to_nhop_subgraphs, 1)

Compute the map from edge to enclosing-subgraph-nodes double radii for the validation and testing sets



In [ ]:
edge_to_double_radii_val_pos = get_double_radii(data.val_pos_edge_index.T.numpy(), node_to_nhop_subgraphs, 1)

edge_to_double_radii_val_neg = get_double_radii(data.val_neg_edge_index.T.numpy(), node_to_nhop_subgraphs, 1)

edge_to_double_radii_test_pos = get_double_radii(data.test_pos_edge_index.T.numpy(), node_to_nhop_subgraphs, 1)

edge_to_double_radii_test_neg = get_double_radii(data.test_neg_edge_index.T.numpy(), node_to_nhop_subgraphs, 1)

## Converting Networkx subgraphs back to Data Objects
Let's recap what we have now. For each edge in the training set, we have extracted an enclosing subgraph. For each node we assigned a structural label called the "double radius". Now we need to translate each of these subgraphs to a list of PyG Data objects. We will also assign a feature vector made by concatenating a one-hot encoding of the double-radius to the original features in the dataset. We will record the existance or non-existance of an edge by assigning a label to these Data objects. (Note this may take some time to complete)

In [ ]:
# Helper function to create the dataset
from torch_geometric.utils import subgraph
from sklearn.preprocessing import OneHotEncoder
from tqdm import *

def create_dataset(edge_to_double_radii_pos, edge_to_double_radii_neg, 
                   max_radius, edge_index, device):
    # One-hot encoding to the maximum radius
    X = [[i] for i in range(max_radius + 1)] 
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(X)

    dataset = []

    for graph_label, edge_to_double_radii in [(0, edge_to_double_radii_neg),
                                        (1, edge_to_double_radii_pos)]:
        for edge in tqdm(edge_to_double_radii):

            double_radii_subgraph = edge_to_double_radii[edge] 
            node_ids_subgraph = sorted(double_radii_subgraph.keys())

            # Create subgraph, with nodes relabed to be contiguous
            edge_index_subgraph,_ = subgraph(node_ids_subgraph, edge_index,
                                            relabel_nodes=True)

            # Convert dict to np.array
            double_radii_subgraph = np.asarray([double_radii_subgraph[key] 
                                                for key in node_ids_subgraph])

            # Create one-hot encoding of the double-radii.
            struct_features = encoder.transform(double_radii_subgraph.reshape(-1,1))

            # Concatenate the one-hot encoding with the existing features of the graph
            x= torch.cat([torch.tensor(struct_features,dtype=torch.float), 
                        data.x[node_ids_subgraph]],dim=1)

            dataset.append(Data(x=x.to(device), edge_index=edge_index_subgraph.to(device), 
                            y=torch.tensor([graph_label]).to(device)).to(device))
    return dataset

In [ ]:
dataset = create_dataset(edge_to_double_radii_train_pos, edge_to_double_radii_train_neg, 2, data_pos.edge_index, 'cuda')


# References

[1] David Liben-Nowell and Jon Kleinberg. The link-prediction problem for social networks. Journal of the
American society for information science and technology, 58(7):1019–1031, 2007.

[2] Lada A Adamic and Eytan Adar. Friends and neighbors on the web. Social networks, 25(3):211–230,
2003.

[3] Yehuda Koren, Robert Bell, and Chris Volinsky. Matrix factorization techniques for recommender systems.
Computer, (8):30–37, 2009.

[4] Maximilian Nickel, Kevin Murphy, Volker Tresp, and Evgeniy Gabrilovich. A review of relational machine
learning for knowledge graphs. Proceedings of the IEEE, 104(1):11–33, 2016.

[5] Tolutola Oyetunde, Muhan Zhang, Yixin Chen, Yinjie Tang, and Cynthia Lo. Boostgapfill: Improving
the fidelity of metabolic network reconstructions through integrated constraint and pattern-based methods.
Bioinformatics, 2016.

[6] Joan Bruna, Wojciech Zaremba, Arthur Szlam, and Yann LeCun. Spectral networks and locally connected
networks on graphs. arXiv preprint arXiv:1312.6203, 2013.

[7] David K Duvenaud, Dougal Maclaurin, Jorge Iparraguirre, Rafael Bombarell, Timothy Hirzel, Alán
Aspuru-Guzik, and Ryan P Adams. Convolutional networks on graphs for learning molecular fingerprints.
In Advances in neural information processing systems, pages 2224–2232, 2015.

[8] Thomas N Kipf and Max Welling. Semi-supervised classification with graph convolutional networks.
arXiv preprint arXiv:1609.02907, 2016.

[9] Mathias Niepert, Mohamed Ahmed, and Konstantin Kutzkov. Learning convolutional neural networks for
graphs. In International conference on machine learning, pages 2014–2023, 2016.

[10] Muhan Zhang, Zhicheng Cui, Marion Neumann, and Yixin Chen. An end-to-end deep learning architecture
for graph classification. In AAAI, pages 4438–4445, 2018.
